In [45]:
import json
import requests
tokenKey = "1234567890abcd"  #14 digits token Key from  https://unwiredlabs.com

In [46]:
##########  decoding form UDA of Sh interface start from here ########
import base64
mccmnc = ''
EpsLocation = 'E0BAVfAQ' # from Sh-User Data in XL EpsLocationInformation
str = base64.b64decode(EpsLocation)
res = ''.join(format(x, '02x') for x in str)

In [47]:
import re
OldMccmnc =  res[0:6]
s = re.findall('..',OldMccmnc)
for es in s:
 #print (es[::-1], end = '')
 mccmnc = mccmnc + es[::-1]

mcc=mccmnc[0:3]
mnc=re.sub("[\D]","",mccmnc[3:6])
cid=int(res[6:],16)

In [48]:
response = requests.post('https://ap1.unwiredlabs.com/v2/process.php',
                   json={"token": tokenKey,"radio": "lte","mcc": mcc,"mnc": mnc,"cells": 
                [{"lac": 1,"cid": cid,"psc": 0}],"address": 1})

In [49]:
print("Status code: ", response.status_code)
print("Printing Entire Post Request")
print(response.json())

Status code:  200
Printing Entire Post Request
{'status': 'ok', 'balance': 4993, 'lat': -6.181257, 'lon': 106.822815, 'accuracy': 437, 'message': 'This cell tower was not found in OpenCelliD. However, we served a location from the Unwired Labs LocationAPI (unwiredlabs.com), an Enterprise Geolocation service with over 100 million cell towers.', 'address': 'Jalan M. Haji Thamrin, RW 04, Gambir, Central Jakarta, Jakarta Special Capital Region, 10110, Indonesia'}


In [50]:
print(response.json()['lat'])

-6.181257


In [51]:

from ipyleaflet import *
from ipywidgets import HTML

In [52]:
m = Map(center=(response.json()['lat'],response.json()['lon']), zoom=100, basemap=basemaps.Stamen.Terrain   )

In [53]:
circle_marker = CircleMarker()
circle_marker.location = (response.json()['lat'],response.json()['lon'])
circle_marker.radius = 100 #response.json()['accuracy']
circle_marker.color = "red"
#circle_marker.fill_color = "red"

marker1 = Marker(location=(response.json()['lat'],response.json()['lon']))

m.add_layer(circle_marker)
m.add_layer(marker1)

message2 = HTML()
message2.value = response.json()['address']

# Popup with a given location on the map:
popup = Popup(
    close_button=False,
    auto_close=False,
    close_on_escape_key=False
)
m.add_layer(popup)
# Popup associated to a layer

marker1.popup = message2

m

Map(center=[-6.181257, 106.822815], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…